# Recurrent Neural Networks (RNN)
Typical machine learning algorithms for supervised learning assume that the input is independent and identically distributed (IID) data, which means that the training examples are mutually independent and have the same underlying distribution. In this regard, based on the mutual independence assumption, the order in which the training examples are given to the model is irrelevant. For example, if we have a sample consisting of n training examples, x(1), x(2), ..., x(n), the order in which we use the data for training our machine learning algorithm does not matter.

What makes sequences unique, compared to other types of data, is that elements in a sequence appear in a certain order and are not independent of each other, i.e., order matters. Predicting the market value of a particular stock would be an example of this scenario.

## Sequential data vs time series data
Time series data is a special type of sequential data where each example is associated with a dimension for time. In time series data, samples are taken at successive timestamps, and therefore, the time dimension determines the order among the data points. For example, stock prices and voice or speech records are time series data.

On the other hand, not all sequential data has the time dimension. For example, in text data or DNA sequences, the examples are ordered, but text or DNA does not qualify as time series data.

## Representing seqeunces
In the ML context, both features and labels are indexed by time as $(\mathbf{x}^{(1)}, \mathbf{x}^{(2)},\cdots, \mathbf{x}^{(T)})$ and $(y^{(1)}, y^{(2)},\cdots, y^{(T)})$. RNNs are designed for modeling sequences and are capable of remembering past information (they have memory) and processing new events accordingly, which is a clear advantage when working with sequence data. 

## Different categrories of sequence data
If either the input or output is a sequence, the modelin task falls into one of these categors:

(Many-to-one)  Input: sequence, Output: not sequence. Exampel: Classifying whether a person liked a movie based on her comment.

(One-to-many) Input: not sequcne, output: sequence, Example: Image (not sequence) captioning (seqeunce)

(Many-to-many) Both sequence. Example: translation

In an RNN with hidden-to-hidden recurrence, the hidden layer receives its input from both the input layer of the current time step and the hidden layer from the previous time step. Since, in this case, each recurrent layer must receive a sequence as input, all the recurrent layers except the last one must return a sequence as output (that is, we will later have to set "return_sequences=True"). The behavior of the last recurrent layer depends on the type of problem.

## Computing activations in the hidden-to-hidden recurrence
We should note that weight matrices in RNN do not depend on time $t$, i.e., they are shared across time axis. Computing the activation is very much similar to standard multilayers perceptron:
\begin{align}
\mathbf{z}^{(t)}_h &= W_{xh}\mathbf{x}^{(t)} + W_{hh}\mathbf{h}^{(t-1)} + \mathbf{b}_h \\
\mathbf{h}^{(t)} &= \sigma_h(\mathbf{z}^{(t)}_h) 
\end{align}
The activation of the output unit can be computed as
\begin{align}
\mathbf{o}^{(t)} = \sigma_0(W_{ho}h^{(t)} + \mathbf{b}_o )
\end{align}

The basic idea to derive the gradient of the composition function is similar to before, while a bit more complicated. The overall loss, $L$, is the sum of all loss function at $t=1,\cdots,T$:
\begin{align}
L = \sum_{t=1}^T L^{(t)}
\end{align}
Since the loss at time $t$ depends on the hidden units at all previous times $1:t$, the gradient will be calculated as
\begin{align}
\frac{\partial L^{(t)}}{W_{hh}} = \frac{\partial L^{(t)}}{o^{(t)}} \times \frac{\partial o^{(t)}}{h^{(t)}} \times (\sum_{k=1}^t \frac{\partial h^{(t)}}{\partial h^{(k)}} \times \frac{\partial h^{(k)}}{W_{hh}})
\end{align}
where 
\begin{align}
\frac{\partial h^{(t)}}{\partial h^{(k)}}  = \Pi_{i=k+1}^t \frac{\partial h^{(i)}}{\partial h^{(i-1)}}
\end{align}

## RNN's output in practice
To see how RNN's output is calculated in practice, let’s manually compute the forward pass for one of these recurrent types. In the following code, we will create a recurrent layer from RNN and perform a forward pass on an input sequence of length 3 to compute the output. We will also manually compute the forward pass and compare the results with those of RNN.


In [1]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True) 

w_xh = rnn_layer.weight_ih_l0 #l0 denoted the first layer
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


In [2]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

print(x_seq)

## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5))) # the feature dimesnion (5 here) should be the last dimension of the tensor for rnn_layer

## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())
    
    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh    
    print('   Hidden          :', ht.detach().numpy())             # Tensor.detach() method in PyTorch is used to separate a tensor from the computational graph by returning a new tensor that doesn’t require a gradient
    
    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()
    

tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]])
Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[-0.4701929  0.5863904]]
   Output (manual) : [[-0.3519801   0.52525216]]
   RNN output      : [[-0.3519801   0.52525216]]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[-0.88883156  1.2364397 ]]
   Output (manual) : [[-0.68424344  0.76074266]]
   RNN output      : [[-0.68424344  0.76074266]]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[-1.3074701  1.886489 ]]
   Output (manual) : [[-0.8649416   0.90466356]]
   RNN output      : [[-0.8649416   0.90466356]]



## The challenges of learning long-range interactions
Back propagation through time introduces some new challenges. Because of the multiplicative factor, in computing the gradients of a loss function, the so-called "vanishing and exploding gradient" problems arise. Basically, $\partial h^{(t)}/\partial h^{(k)}$ has $t-k$ multiplications; therefore, multiplying the weight, $W$, by itself $t – k$ times. As a result, if $|W| < 1$, this factor becomes very small when $t – k$ is large. On the other hand, if the weight of the recurrent edge is $|W| > 1$, then $W^{t–k}$ becomes very large when t – k is large. Note that a large t – k refers to long-range dependencies.

Solutions: 1. Gradient clipping, 2. Truncated backpropagation through time, 3. "LSTM" (see the slides)

## Sentiment analysis 
We will implement many-to-one architecture for IMDB movie review data sentiment analysis. First, we will import the data from "torchtext" and do some data preprocessing:

In [3]:
import torch
import torch.nn as nn

# conda install -c pytorch torchtext

from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split


In [18]:
# features are sequence of words, labels are "neg"/0 (negative sentiment) and "pos"/1 (positive sentiment)

## Step 1: load and create the datasets

# conda install -c conda-forge portalocker

train_dataset = IMDB(split='train')  # 25,000 samples
test_dataset = IMDB(split='test')   # 25,000 samples

torch.manual_seed(1)
train_dataset, valid_dataset = random_split(list(train_dataset), [20000, 5000]) # split training set into 20,000+5,000

To prepare the data for input to an NN, we need to encode it into numeric values (steps 2 and 3 below). To do this, we will first find the unique words (tokens) in the training dataset. While finding unique tokens is a process for which we can use Python datasets, it can be more efficient to use the Counter class from the collections package, which is part of Python’s standard library.

In the following code, we will instantiate a new "Counter object" (token_counts) that will collect the unique word frequencies. Note that in this particular application (and in contrast to the bag-of-words model), we are only interested in the set of unique words and won’t require the word counts, which are created as a side product. To split the text into words (or tokens), we will use the "tokenizer function", which also removes HTML markups as well as punctuation and other non-letter characters:

In [5]:
## Step 2: find unique tokens (words)

import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

Vocab-size: 69023


Next, we are going to map each unique word to a unique integer. This can be done manually using a Python dictionary, where the keys are the unique tokens (words) and the value associated with each key is a unique integer. However, the torchtext package already provides a class, Vocab, which we can be used to create such a mapping and encode the entire dataset. First, we will create a vocab object by passing the ordered dictionary mapping tokens to their corresponding occurrence frequencies (the ordered dictionary is the sorted token_counts). Second, we will prepend two special tokens to the vocabulary – the padding and the unknown token. Finally, to demonstrate how to use the vocab object, we will convert an example input text into a list of integer values.

In [6]:
## Step 3: encoding each unique token into integers
from torchtext.vocab import vocab

sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vocab = vocab(ordered_dict) # creates the integer mapping

vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['this', 'is', 'an', 'example']])

[11, 7, 35, 457]


Note that there might be some tokens in the validation or testing data that are not present in the training data and are thus not included in the mapping. If we have q tokens (that is, the size of token_counts passed to Vocab, which in this case is 69,023), then all tokens that haven’t been seen before, and are thus not included in token_counts, will be assigned the integer 1 (a placeholder for the unknown token). In other words, the index 1 is reserved for unknown words. Another reserved value is the integer 0, which serves as a placeholder, a so-called padding token, for adjusting the sequence length.

We can define the text_pipeline function to transform each text in the dataset accordingly and the label_pipeline function to convert each label to 1 or 0:

In [7]:
## Step 3-A: define the functions for transformation

# device = torch.device("cuda:0")  # If the system has gpu
device = 'cpu'

text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1. if x == 'pos' else 0.

We will generate batches of samples using DataLoader and pass the data processing pipelines declared previously to the argument collate_fn. We will wrap the text encoding and label transformation function into the collate_batch function:

In [8]:
## Step 3-B: wrap the encode and transformation function
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), 
                                      dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(text_list, batch_first=True) # pad_sequence is discussed below
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

So far, we’ve converted sequences of words into sequences of integers, and labels of pos or neg into 1 or 0. However, there is one issue that we need to resolve—the sequences currently have different lengths (as shown in the result of executing the following code for four examples). Although, in general, RNNs can handle sequences with different lengths, we still need to make sure that all the sequences in a mini-batch have the same length to store them efficiently in a tensor.

PyTorch provides an efficient method, "pad_sequence()", which will automatically pad the consecutive elements that are to be combined into a batch with placeholder values (0s) so that all sequences within a batch will have the same shape. In the previous code, we already created a data loader of a small batch size from the training dataset and applied the collate_batch function, which itself included a pad_sequence() call.

However, to illustrate how padding works, we will take the first batch and print the sizes of the individual elements before combining these into mini-batches, as well as the dimensions of the resulting mini-batches: 

In [9]:
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[   35,  1739,     7,   449,   721,     6,   301,     4,   787,     9,
             4,    18,    44,     2,  1705,  2460,   186,    25,     7,    24,
           100,  1874,  1739,    25,     7, 34415,  3568,  1103,  7517,   787,
             5,     2,  4991, 12401,    36,     7,   148,   111,   939,     6,
         11598,     2,   172,   135,    62,    25,  3199,  1602,     3,   928,
          1500,     9,     6,  4601,     2,   155,    36,    14,   274,     4,
         42945,     9,  4991,     3,    14, 10296,    34,  3568,     8,    51,
           148,    30,     2,    58,    16,    11,  1893,   125,     6,   420,
          1214,    27, 14542,   940,    11,     7,    29,   951,    18,    17,
         15994,   459,    34,  2480, 15211,  3713,     2,   840,  3200,     9,
          3568,    13,   107,     9,   175,    94,    25,    51, 10297,  1796,
            27,   712,    16,     2,   220,    17,     4,    54,   722,   238,
           395,     2,   787,    32,    27,  5236,  

Finally, let’s divide all three datasets into data loaders with a batch size of 32:

In [19]:
## Step 4: batching the datasets

batch_size = 32  

train_dl = DataLoader(train_dataset, batch_size=batch_size,
                      shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,
                      shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)

In the next subsection, however, we will first discuss feature embedding, which is an optional but highly recommended preprocessing step that is used to reduce the dimensionality of the word vectors.

### Embedding layers for sentence encoding
We map each word to a vector of a fixed size with real-valued elements (not necessarily integers). Given the number of unique words, nwords, we can select the size of the embedding vectors (a.k.a., embedding dimension) to be much smaller than the number of unique words (embedding_dim << nwords) to represent the entire vocabulary as input features. The embedding matrix serves as the input layer to our NN models. In practice, creating an embedding layer can simply be done using nn.Embedding. Note that the padding tokens will not contribute to the gradient in updates.

In [11]:
import torch
import torch.nn as nn

embedding = nn.Embedding(num_embeddings=10,   # size of the dictionary 
                         embedding_dim=3, 
                         padding_idx=0)
 
# a batch of 2 samples (sentences) of 4 tokens (words) each
text_encoded_input = torch.LongTensor([[1,2,4,5],[4,3,2,0]])
print(embedding(text_encoded_input))

tensor([[[ 0.7039, -0.8321, -0.4651],
         [-0.3203,  2.2408,  0.5566],
         [-0.4643,  0.3046,  0.7046],
         [-0.7106, -0.2959,  0.8356]],

        [[-0.4643,  0.3046,  0.7046],
         [ 0.0946, -0.3531,  0.9124],
         [-0.3203,  2.2408,  0.5566],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)


### Building an RNN model
Using "nn.Module" we can combine the embedding layer, the recurrent layer, and the fully connected non-recurrent layers. For the recurrent layer, we can use RNN, LSTM, or GRU (multi-layer Gated Recurrent Unit). In the example below, we use two recurrent layers of RNN + a non-recurent fully connected (fc) layer + an output layer:

In [12]:
## An example of building a RNN model
## with simple RNN layer

# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):   # hidden_size = number of rnn cells in each hidden layer
        super().__init__()
        self.rnn = nn.RNN(input_size, 
                          hidden_size, 
                          num_layers=2, 
                          batch_first=True)
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)           #multi-layer gated recurrent unit (GRU) RNN
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)   # fc stands for fully connected
        
    def forward(self, x):
        _, hidden = self.rnn(x)
        out = hidden[-1, :, :]
        out = self.fc(out)
        return out

model = RNN(64, 32) 

print(model) 
 
model(torch.randn(5, 3, 64))    # input: (batch_size, seq_len, features)     output: (batch_size, output_size)

# batch_size = number of texts (each text is potentially multiple sentences) per batch, seq_len = number of tokens (words)  per text, features = embedding dimension

RNN(
  (rnn): RNN(64, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


tensor([[ 0.3183],
        [ 0.1230],
        [ 0.1772],
        [-0.1052],
        [-0.1259]], grad_fn=<AddmmBackward0>)

If your input data is of shape (seq_len, batch_size, features) then you don’t need batch_first=True and your LSTM will give output of shape (seq_len, batch_size, hidden_size).

If your input data is of shape (batch_size, seq_len, features) then you need batch_first=True and your LSTM will give output of shape (batch_size, seq_len, hidden_size).

### Building an RNN model for sentiment analysis
Since we have very long sequences, we are going to use an LSTM layer to account for long-range effects. We will create an RNN model for sentiment analysis, starting with an embedding layer producing word embeddings of feature size 20 (embed_dim=20). Then, a recurrent layer of type LSTM will be added. Finally, we will add a fully connected layer as a hidden layer and another fully connected layer as the output layer, which will return a single class-membership probability via the sigmoid activation function.

In [13]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,              
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        out, (hidden, cell) = self.rnn(out)  # hidden = (num_layers * num_directions, batch, hidden_size)
        out = hidden[-1, :, :] # get the last hidden state
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
         
vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64 # hidden_size = number of rnn cells in each hidden layer
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device) # moves the model to the decice (cpu or gpu)

Now, we train the model for one apoch and return the classification accuracy and loss:

In [14]:
def train(dataloader):
    model.train() # set the model to training mode
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

we next develop the evaluate function to measure the model’s performance on a given dataset:

In [15]:
def evaluate(dataloader):
    model.eval() # set the model to evaluation mode
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

The next step is to create a loss function and optimizer (Adam optimizer). For a binary classification with a single class-membership probability output, we use the binary cross-entropy loss (BCELoss) as the loss function:

In [16]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2 

torch.manual_seed(1) 

for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

Epoch 0 accuracy: 0.9987 val_accuracy: 1.0000
Epoch 1 accuracy: 1.0000 val_accuracy: 1.0000


After training this model for 10 epochs, we will evaluate it on the test data:

In [20]:
acc_test, _ = evaluate(test_dl)
print(f'test_accuracy: {acc_test:.4f}') 

TypeError: _IterDataPipeSerializationWrapper instance doesn't have valid length

### Bidirectional RNN
We will set the bidirectional configuration of the LSTM to True, which will make the recurrent layer pass through the input sequences from both directions, start to end, as well as in the reverse direction:

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True, bidirectional=True)   # bidirectional=True
        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)
        _, (hidden, cell) = self.rnn(out)
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
    
torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)

The bidirectional RNN layer makes two passes over each input sequence: a forward pass and a reverse or backward pass (note that this is not to be confused with the forward and backward passes in the context of backpropagation). The resulting hidden states of these forward and backward passes are usually concatenated into a single hidden state. Other merge modes include summation, multiplication (multiplying the results of the two passes), and averaging (taking the average of the two).

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

num_epochs = 2 

torch.manual_seed(1)
 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

In [ ]:
test_dataset = IMDB(split='test')
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)

In [ ]:
acc_test, _ = evaluate(test_dl)
print(f'test_accuracy: {acc_test:.4f}') 